In [14]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [2]:
df = pd.read_csv("data/deviatoric_concatenate.csv")

In [3]:
df.describe()

,velocity:0,velocity:1,force:0,force:1,total displacement:0,total displacement:1,temperature,temp_power,temp_pressure,temp_density,...,stress XZ,density,thermal_stress,energy_total_vol_dev,energy_volumetric,energy_deviatoric,energy_thermal,energy_elastic,coordinate:0,coordinate:1
count,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,...,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06,2.660160e+06
mean,6.329685e-11,3.727487e-11,-1.367216e+07,-7.402520e+06,1.453043e+03,3.305589e+03,6.158631e+00,4.609019e+04,-3.974861e+01,3.913310e-06,...,1.629498e+06,2.405160e+03,-9.380253e+06,2.722995e+08,-5.729299e+06,2.894874e+08,1.365979e+12,6.663074e+10,5.238703e+04,-2.527535e+03
std,1.550915e-10,8.895128e-11,2.029575e+09,2.357488e+09,9.714274e+03,3.906848e+03,8.942963e+00,5.657852e+05,2.920473e+04,2.309945e-04,...,1.722122e+07,3.368318e+02,1.364452e+07,3.999082e+08,9.904921e+06,4.220148e+08,7.914640e+12,7.909412e+11,2.175735e+04,3.923942e+03
min,-1.216000e-09,-9.896700e-10,-1.422800e+11,-1.095100e+11,-2.022600e+04,-1.443700e+03,-1.722400e-07,0.000000e+00,-1.453300e+07,-1.318800e-01,...,-1.508600e+08,1.231500e+03,-7.650600e+07,0.000000e+00,-4.503100e+07,0.000000e+00,0.000000e+00,-8.275800e+11,-1.995000e+04,-1.010000e+04
25%,-8.732425e-11,-5.109225e-12,-2.336600e+05,-1.969700e+05,-5.646900e+03,2.327675e+01,3.263000e-01,0.000000e+00,-6.587000e+01,-1.287100e-07,...,-5.402000e+06,2.291900e+03,-1.347400e+07,7.782900e+05,-4.502800e+06,9.726975e+05,7.997400e+09,-5.844600e+08,3.791800e+04,-5.496600e+03
50%,1.234500e-10,4.716500e-12,3.800000e+04,2.665700e+04,1.715300e+03,1.877300e+03,1.390500e+00,0.000000e+00,1.347150e-01,4.324400e-15,...,5.371100e+05,2.472900e+03,-2.167850e+06,5.333100e+07,-8.136900e+05,5.853300e+07,9.985400e+10,3.198300e+08,5.094200e+04,-2.150750e+03
75%,1.765500e-10,7.329900e-11,3.667400e+05,2.745500e+05,9.275500e+03,5.382100e+03,8.941200e+00,0.000000e+00,8.885300e+01,1.509600e-07,...,7.358400e+06,2.712900e+03,-5.628100e+05,4.407300e+08,-1.629000e+04,4.658000e+08,7.281225e+11,6.932400e+09,6.424300e+04,6.702600e+02
max,1.245200e-09,7.588300e-10,2.041800e+11,1.354200e+11,2.001300e+04,1.537800e+04,5.257800e+01,1.240300e+08,1.518800e+07,1.049500e-01,...,1.522300e+08,2.726400e+03,1.209600e+06,1.972000e+09,8.682900e+05,2.023400e+09,7.434200e+14,6.861700e+13,1.199500e+05,5.642000e+03


In [4]:
df.columns

Index(['velocity:0', 'velocity:1', 'force:0', 'force:1',
       'total displacement:0', 'total displacement:1', 'temperature',
       'temp_power', 'temp_pressure', 'temp_density', 'plastic strain',
       'plastic strain-rate', 'strain-rate II log10', 'strain-rate XX',
       'elastic_strain XX', 'strain-rate ZZ', 'elastic_strain ZZ',
       'strain-rate XZ', 'elastic_strain XZ', 'strain I', 'strain II',
       'strain XX', 'strain ZZ', 'strain XZ', 'stress I', 'stress II',
       'stress XX', 'stress ZZ', 'stress XZ', 'density', 'thermal_stress',
       'energy_total_vol_dev', 'energy_volumetric', 'energy_deviatoric',
       'energy_thermal', 'energy_elastic', 'coordinate:0', 'coordinate:1'],
      dtype='object')

In [5]:
df.describe()['plastic strain']

count    2.660160e+06
mean     3.095186e+00
std      4.112005e+00
min      0.000000e+00
25%      4.768800e-03
50%      3.100500e-01
75%      5.900300e+00
max      2.176800e+01
Name: plastic strain, dtype: float64

In [6]:
threshold = 0.15
df['plastic strain'] = np.where(df['plastic strain'] > threshold, 1,0)

In [7]:
dropped_cols = ['plastic strain', 'plastic strain-rate', 'strain-rate II log10', 'strain-rate XX',
       'elastic_strain XX', 'strain-rate ZZ', 'elastic_strain ZZ',
       'strain-rate XZ', 'elastic_strain XZ', 'strain I', 'strain II',
       'strain XX', 'strain ZZ', 'strain XZ', 'coordinate:0', 'coordinate:1', 'force:0', 'force:1', 
       'total displacement:0', 'total displacement:1', 'stress I', 'stress II', 'energy_volumetric', 
       'energy_deviatoric','energy_thermal', 'energy_elastic', 'thermal_stress',
       'energy_total_vol_dev', 'temp_power', 'temp_pressure', 'temp_density']

Y = df['plastic strain']
df = df.drop(dropped_cols, axis=1)
X = df

In [8]:
df.columns

Index(['velocity:0', 'velocity:1', 'temperature', 'stress XX', 'stress ZZ',
       'stress XZ', 'density'],
      dtype='object')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [10]:
mms = StandardScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.fit_transform(X_test)

In [11]:
class Create_ensemble(object):
    def __init__(self, n_splits, base_models):
        self.n_splits = n_splits
        self.base_models = base_models
        
    def predict(self, X, y, x_test, y_test):
        X = np.array(X)
        y = np.array(y)
        x_test = np.array(x_test)

        folds = list(KFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((x_test.shape[0], len(self.base_models)))
        
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((x_test.shape[0], self.n_splits))

            for j, (train_idx, valid_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_valid = X[valid_idx]
                y_valid = y[valid_idx]
                clf.fit(X_train, y_train)
                valid_pred = clf.predict(X_valid)
                S_train[valid_idx, i] = valid_pred
                test_pred = clf.predict(x_test)
                S_test_i[:, j] = test_pred
                
                print('Model: {}, fold: {}, train_acc: {},  test_acc: {}'.format(i, j, accuracy_score(y_valid, valid_pred), accuracy_score(y_test, test_pred)))
                
            print( "\nTraining RMSLE for model {} : {}".format(i, accuracy_score(y, S_train[:,i])))
            S_test[:, i] = S_test_i.mean(axis=1)
            
        return S_train, S_test, clf

In [15]:
rfc = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=6, verbose=0, warm_start=False)

In [16]:
stack = Create_ensemble(n_splits = 10, base_models = [rfc])

X = X_train
Y = y_train

train_pred, test_pred, model = stack.predict(X, Y, X_test, y_test)

Model: 0, fold: 0, train_acc: 0.9769402616372737,  test_acc: 0.9774725329804723
Model: 0, fold: 1, train_acc: 0.9771282194488002,  test_acc: 0.9773735414411163
Model: 0, fold: 2, train_acc: 0.9774073497269227,  test_acc: 0.9774549901760295
Model: 0, fold: 3, train_acc: 0.9774019794748967,  test_acc: 0.9775063655318978
Model: 0, fold: 4, train_acc: 0.9771656883857559,  test_acc: 0.9774098801074622
Model: 0, fold: 5, train_acc: 0.9773912389708449,  test_acc: 0.977408627050002
Model: 0, fold: 6, train_acc: 0.9772730934262744,  test_acc: 0.977513883876659
Model: 0, fold: 7, train_acc: 0.9768810650283818,  test_acc: 0.9775264144512611
Model: 0, fold: 8, train_acc: 0.9773053149384301,  test_acc: 0.9773547455792133
Model: 0, fold: 9, train_acc: 0.9780141882058525,  test_acc: 0.9775001002445968

Training RMSLE for model 0 : 0.977290839648743


In [17]:
Y_pred = model.predict(X_test)

In [18]:
print('The accuracy of the model is {}'.format(accuracy_score(y_test, Y_pred)))
print('# Classification report \n {}'.format(classification_report(y_test, Y_pred)))

The accuracy of the model is 0.9775001002445968
# Classification report 
              precision    recall  f1-score   support

          0       0.99      0.96      0.97    351678
          1       0.97      0.99      0.98    446370

avg / total       0.98      0.98      0.98    798048



In [19]:
def plot_important_features(df, model):
    values = sorted(zip(df.columns, model.feature_importances_), key=lambda x: x[1] * -1)
    sorted_features, scores = zip(*values)

    trace0 = go.Bar(
        x= sorted_features,
        y= scores*100,
    #     text=['27% market share', '24% market share', '19% market share'],
        marker=dict(
            color='rgb(158,202,225)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5,
            )
        ),
        opacity=0.6
    )

    data = [trace0]
    layout = go.Layout(
        title='Important features',
    )

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='text-hover-bar')

In [20]:
plot_important_features(df, model)